In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os.path
import numpy as np
import pandas as pd
import dateutil
import datetime
import re
import matplotlib.pyplot as plt

import load_data

# pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
#test

In [ ]:
b = 'Capacity test Raw data- 2017-03-25_7 PM.csv'
c = 'Data_Extract_20170807-20170807_cap_test_daily_id38778c9c-115.csv'
raw_data = load_data.load_das_data('./data/', b)
#raw_data.columns.tolist()

In [ ]:
raw_data.shape

In [ ]:
col_types = raw_data.apply(load_data.series_type, args=(load_data.type_defs,)).tolist()
sub_types = raw_data.apply(load_data.series_type, args=(load_data.sub_type_defs,), bounds_check=False).tolist()
irr_types = raw_data.apply(load_data.series_type, args=(load_data.irr_sensors_defs,), bounds_check=False).tolist()

col_indices = ['-'.join([typ, sub_typ, irr_typ]) for typ, sub_typ, irr_typ in zip(col_types, sub_types, irr_types)]

raw_data_clean_headers = raw_data.copy()
raw_data_clean_headers.columns = col_indices

eq_cnts = load_data.equip_counts(raw_data_clean_headers)

In [ ]:
len(raw_data_clean_headers.columns.tolist())

In [ ]:
len(eq_cnts)

In [ ]:
n_col_names = list(zip(col_indices, eq_cnts))
raw_data_clean_headers.columns = pd.MultiIndex.from_tuples(n_col_names, names=['label','equip_cnt'])

In [ ]:
raw_data_clean_headers.head()

In [ ]:
#(raw_data_clean_headers['temp-amb-'][1] - raw_data_clean_headers['temp-amb-'][2]) / raw_data_clean_headers['temp-amb-'][1]
df = raw_data_clean_headers['irr-poa-ref_cell']

In [ ]:
df.shape

In [ ]:
def sensor_filter(df, perc_diff):
    if df.shape[1] > 2:
        return df[df.apply(std_filter, axis=1)].index
    elif df.shape[1] == 1:
        return df
    else:
        # should return the index and take the intersection for all the types and filter entire df on that
        return df[(df.iloc[:,0] - df.iloc[:,1])/df.iloc[:,0] < perc_diff].index

In [ ]:
def std_filter(series):
    mean = series.mean()
    std = series.std()
    min_bound = mean - std*2
    max_bound = mean + std*2
    return all(series.apply(lambda x: min_bound < x < max_bound))

In [ ]:
sens_filt = sensor_filter(df, 0.05)

In [ ]:
sens_filt.shape

In [ ]:
df_temp_mod = raw_data_clean_headers['temp-mod-']

In [ ]:
sens_filt_mod = sensor_filter(df_temp_mod, 0.05)

In [ ]:
sens_filt_mod.shape